# Movie Recommendation System

The goal of this project is to build a movie recommendation system using the TMDB dataset. In an era of overwhelming content choices, helping users discover movies that align with their tastes is both a practical and valuable application of machine learning. Recommender systems are essential in improving user engagement and satisfaction across entertainment platforms such as Netflix, Prime Video, and TMDB itself.

Kaggle Dataset: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

## 1. Load Libraries and Dataset

In [1]:
# --- Install Dependecies ---
! pip install opendatasets

In [2]:
# --- Load Dataset ---

import opendatasets
opendatasets.download('https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata')

Skipping, found downloaded files in "./tmdb-movie-metadata" (use force=True to force download)


In [3]:
# --- Import Necessary Libraries ---

import numpy as np
import pandas as pd
import ast
import joblib
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import csr_matrix

# Import Nereast Neighbors model 
from recommender import MovieRecommender

In [4]:
# --- Load Files --- 

movies = pd.read_csv('tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb-movie-metadata/tmdb_5000_credits.csv')

In [5]:
# First instance in movies dataset
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
# First 5 instance in credits dataset
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [7]:
# Dimenstion of movies dataset
movies.shape

(4803, 20)

In [8]:
# Dimension of credits dataset
credits.shape

(4803, 4)

In [9]:
## To simplify, we can add the movie information with the credit information based on title
movies = movies.merge(credits, on='title')

In [10]:
# Check the new dimenstions of movies dataset
movies.shape

(4809, 23)

In [11]:
# First instance of new movies dataset
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


## 2. Explanatory Data Analysis

In [12]:
# Info about each of the columns
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [13]:
# --- Check for null values ---
movies.isnull().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

**Note:** the columns homepage and tagline have many null values. Some of the columns like 'movie_id', 'id', 'budget', 'original_title', 'production_companies', 'production_countries', 'spoken_languages' and 'tagline' does not provide much information to the recommendation system.
On the other hand, 'release_date', 'popularity', 'revenue', 'vote_average' or 'vote_count' might be a good indicator to find similar movies, but they do not provide enough insights to one's taste/preferences.


### 2.1 Feature Engineering

In [14]:
# -- Check the status of movies in the list ---
movies['status'].unique()

array(['Released', 'Post Production', 'Rumored'], dtype=object)

**Note:** movies that are 'Rumored' should not be recommended to the user because they are not released yet.

In [15]:
movies = movies[movies.status != 'Rumored']

In [16]:
movies['status'].unique()

array(['Released', 'Post Production'], dtype=object)

In [17]:
movies.shape

(4804, 23)

**Note:** now that we have cleared all 'Rumored' movies, status is not needed in the movie dataset

In [18]:
# --- Reusable Functions : To dissect the importatnt values from dictionaries ---

# Get the important value from genres, keywords and cast 
def convert_dict(text):
    result = []
    for i in ast.literal_eval(text):
        result.append(i['name'])
    return result

In [19]:
# Get the direcror's name for each movie from cast column
def fetch_director(text):
    result = ""
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            result = i['name']
    return result

In [20]:
movies['genres'] = movies['genres'].apply(convert_dict)
movies['keywords'] = movies['keywords'].apply(convert_dict)
movies['cast'] = movies['cast'].apply(convert_dict)
movies['production_companies'] = movies['production_companies'].apply(convert_dict)
movies['production_countries'] = movies['production_countries'].apply(convert_dict)
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [21]:
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron


In [22]:
# Limit the number of genres, keywords and cast for each movie
movies['cast'] = movies['cast'].apply(lambda x:x[0:5]) # max of 5 cast members
movies['genres'] = movies['genres'].apply(lambda x:x[0:2]) # max of 2 genres
movies['keywords'] = movies['keywords'].apply(lambda x:x[0:5]) # max of 5 keywords
movies['production_companies'] = movies['production_companies'].apply(lambda x:x[0:1]) # max of 1 production company
movies['production_countries'] = movies['production_countries'].apply(lambda x:x[0:1]) # max of 1 production country
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,[Ingenious Film Partners],...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,300000000,"[Adventure, Fantasy]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,[Walt Disney Pictures],...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,245000000,"[Action, Adventure]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi6]",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,[Columbia Pictures],...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,250000000,"[Action, Crime]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,[Legendary Pictures],...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,260000000,"[Action, Adventure]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton


In [23]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

**Note:** Now that we have processed the main columns of the dataset, it is time to choose columns which are useful for this problem set. 
The columns that are useful for movie recommendation based on another movie are genre, cast, director, rating, and keywords (general theme). 

In [24]:
# --- Dimensionality reduction for K-means ---
movies = movies[['id', 'title','genres','cast','vote_average','crew', 'keywords', 'original_language', 'production_companies', 'production_countries']]
movies.rename(columns={'crew': 'director'}, inplace=True)
movies.head()

,id,title,genres,cast,vote_average,director,keywords,original_language,production_companies,production_countries
0,19995,Avatar,"[Action, Adventure]","[Sam Worthington, Zoe Saldana, Sigourney Weave...",7.2,James Cameron,"[culture clash, future, space war, space colon...",en,[Ingenious Film Partners],[United States of America]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy]","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",6.9,Gore Verbinski,"[ocean, drug abuse, exotic island, east india ...",en,[Walt Disney Pictures],[United States of America]
2,206647,Spectre,"[Action, Adventure]","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",6.3,Sam Mendes,"[spy, based on novel, secret agent, sequel, mi6]",en,[Columbia Pictures],[United Kingdom]
3,49026,The Dark Knight Rises,"[Action, Crime]","[Christian Bale, Michael Caine, Gary Oldman, A...",7.6,Christopher Nolan,"[dc comics, crime fighter, terrorist, secret i...",en,[Legendary Pictures],[United States of America]
4,49529,John Carter,"[Action, Adventure]","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",6.1,Andrew Stanton,"[based on novel, mars, medallion, space travel...",en,[Walt Disney Pictures],[United States of America]


### 2.2 Creating Feature Matrix

In [25]:
# --- Creating Dummies ---
def create_dummies(column_name):
    new_columns = movies[column_name]

    mlb = MultiLabelBinarizer()

    return pd.DataFrame(mlb.fit_transform(new_columns),columns=mlb.classes_, index=movies.index)

**Note:** Create dummies to vectorize columns with catagorical values. 

In [26]:
movies = pd.concat([movies, create_dummies('genres')], axis=1)
movies = pd.concat([movies, create_dummies('cast')], axis=1)
movies = pd.concat([movies, create_dummies('director')], axis=1)
movies = pd.concat([movies, create_dummies('keywords')], axis=1)
movies = pd.concat([movies, create_dummies('production_companies')], axis=1)
movies = pd.concat([movies, create_dummies('production_countries')], axis=1)

In [27]:
# Add language dummies
movies = pd.concat([movies, pd.get_dummies(movies['original_language'], dtype=int)], axis = 1)

In [28]:
# Drop irrelevant columns
movies.drop(['id', 'cast', 'genres', 'director', 'keywords', 'original_language', 'production_companies', 'production_countries'], axis=1, inplace=True)

movies.head()

,title,vote_average,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,...,ru,sl,sv,ta,te,th,tr,vi,xx,zh
0,Avatar,7.2,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pirates of the Caribbean: At World's End,6.9,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Spectre,6.3,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Dark Knight Rises,7.6,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,John Carter,6.1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# Make the dataset a Pivot Table
movies = movies.set_index('title')
movies.head()

,vote_average,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,ru,sl,sv,ta,te,th,tr,vi,xx,zh
title,,,,,,,,,,,,,,,,,,,,,
Avatar,7.2,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pirates of the Caribbean: At World's End,6.9,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Spectre,6.3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Dark Knight Rises,7.6,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
John Carter,6.1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# --- Convert the pivot_table into an array matrix ---

# All info of pivot table converted into an array
features_matrix = csr_matrix(movies.values)
features_matrix

<4804x16043 sparse matrix of type '<class 'numpy.float64'>'
	with 120176 stored elements in Compressed Sparse Row format>

## 3. Unsupervised Nearest Neighbors Model

In [31]:
# --- Create an instance of recommnder class (imported from recommender.py file) ----
recommender = MovieRecommender(features_matrix, movies)

# Save the model object as model.pkl
joblib.dump(recommender, "model.pkl")

['model.pkl']

In [32]:
# Try examples
print(recommender.recommend('Avatar'))

['Counting', 'The Abyss', 'Aliens', 'Terminator 2: Judgment Day', 'True Lies']


In [33]:
# Try examples
print(recommender.recommend('The Dark Knight Rises'))

['Batman Begins', 'The Dark Knight', 'Inception', 'The Prestige', 'Interstellar']


In [34]:
print(recommender.recommend('Pirates of the Caribbean: The Curse of the Black Pearl'))

["Pirates of the Caribbean: At World's End", "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Counting', 'Rango']


In [35]:
print(recommender.recommend('Alice in Wonderland'))

['Alice Through the Looking Glass', 'Big Fish', 'Edward Scissorhands', 'Ed Wood', 'Beetlejuice']


In [39]:
print(recommender.recommend('Cars 2'))

['WALL·E', 'The Muppet Christmas Carol', "Pooh's Heffalump Movie", 'Diary of a Wimpy Kid: Dog Days', 'Ratatouille']
